In [1]:
import fitz  #import PyMuPDF module
import os #module used to interact with local Operating System.
import pytesseract
import matplotlib.pyplot as plt
import cv2
os.environ['HF_HOME'] = '/mnt/data/thomas/.cache'
import torch
import transformers


os.environ["LD_LIBRARY_PATH"] = "/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/math_libs/12.4/targets/x86_64-linux/lib/"

## Prompt preparation

### Stage 1

#### Text-gen prompts

In [8]:
prompts_1 = []
system_role = {"role": "system", "content": "You are a helpful academic assistant trained to answer questions about academic policy concisely."}
prompts_1.append({"role": "user", "content": "Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?"})
prompts_1.append({"role": "user", "content": "What are the consequences if I withdraw from capstone II?"})
prompts_1.append({"role": "user", "content": "Can I continue my study for more than 4 years with financial aid?"})
prompts_1.append({"role": "user", "content": "Can you differentiate a cross-listed course and elective applied course in the case of a double major?"})
prompts_1.append({"role": "user", "content": "I would like to ask for the capstone withdrawal policy. What will be the impact when I choose to drop the capstone before the Fall 2025 term starts? Will there be any penalties associated with withdrawing, such as a 'W' notation on my transcript?"})
prompts_1.append({"role": "user", "content": "I have taken Microeconomic Analysis and Macroeconomic Analysis in Fall 2023. Am I eligible to take Financial Economics in Spring 2024?"})

### Stage 2

In [2]:
prompts_2 = []
prompts_2.append({'role': 'user', 'content': 'Context: There are earned credits and attempted credits. Earned credits refer to the number of credit hours a student successfully completes with a grade of D or higher for letter grade options and a Pass for Pass/No Pass options. Attempted credits are the credit hours students enroll in and are recorded in the transcript, regardless of whether they complete the courses or pass them. This includes all courses for which a student receives a grade, including withdrawals (W), incompletes (I), repeats (RP), and failures (F). See section 4 for more details on different types of grades. Question: Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?'})
prompts_2.append({'role': 'user', 'content': 'Context: All majors at Fulbright offer an optional capstone pathway to be completed during the student’s final year. Students may apply for the opportunity to fulfill 8 credits of their major requirements via a capstone project. A student wishing to do a capstone must apply during the term prior to the first term of the capstone (Capstone I), and may only apply after having completed 80 credits of coursework. Question: What are the consequences if I withdraw from capstone II?'})
prompts_2.append({'role': 'user', 'content': 'Context: Fulbright follows MOET regulations, which state that the maximum time to complete the program cannot exceed double the standard duration. Question: Can I continue my study for more than 4 years with financial aid?'})
prompts_2.append({'role': 'user', 'content': 'Context: For Applied Mathematics, Elective Applied Courses (2 required). Students pursuing a double major must complete all requirements for each major. If two majors (or two minors, or a major and a minor) share cross- listed or tagged1 courses, up to 8 credits (two courses) can be used to satisfy both sets of requirements. Question: Can you differentiate a cross-listed course and elective applied course in the case of a double major?'})
prompts_2.append({'role': 'user', 'content': """Context: All majors at Fulbright offer an optional capstone pathway to be completed during the student’s final year. Students may apply for the opportunity to fulfill 8 credits of their major requirements via a capstone project. A student wishing to do a capstone must apply during the term prior to the first term of the capstone (Capstone I), and may only apply after having completed 80 credits of coursework. Students who do not do a capstone will need to fulfill 8 credits of coursework as specified in their major. Individual majors will determine eligibility requirements, and students should discuss their proposals with program coordinators and potential advisors to prepare their applications. Capstone projects will be graded No Pass/Pass/Honors and must be evaluated by the supervising faculty at least one month before the end of the student’s graduating term. Students whose capstone is graded Honors will be awarded Graduation with Honors, answer this question: I would like to ask for the capstone withdrawal policy. What will be the impact when I choose to drop the capstone before the Fall 2025 term starts? Will there be any penalties associated with withdrawing, such as a 'W' notation on my transcript?"""})
prompts_2.append({'role': 'user', 'content': 'Context: Econometrics: This course is concerned with the application of statistical theory to the analysis of economic data and the estimation of economic relationships. The course focuses on regression analysis and its uses in empirical economic research. Students will learn how to construct economic models and test them with data. Microeconomic Analysis: This course focuses on how incentives both constrain and direct the decision making of consumers, producers, and governments. Students will learn to use both graphical and optimization techniques to solve the problems faced by consumers (what to buy), producers (what to produce and what price to sell it at), and governments (which policies to enact). Macroeconomic Analysis: In this course, students will combine empirical observations and economic models to study the dynamics of the aggregate economy. This course focuses on the macroeconomic tools of government – fiscal and monetary policy – and their effects on long-run economic growth, employment, and inflation. Question: I have taken Microeconomic Analysis and Macroeconomic Analysis in Fall 2023. Am I eligible to take Financial Economics in Spring 2024?'})

### Stage 3

In [3]:
prompts_3 = []
prompts_3.append({'role': 'user', 'content': 'Context: There are earned credits and attempted credits. Earned credits refer to the number of credit hours a student successfully completes with a grade of D or higher for letter grade options and a Pass for Pass/No Pass options. Attempted credits are the credit hours students enroll in and are recorded in the transcript, regardless of whether they complete the courses or pass them. This includes all courses for which a student receives a grade, including withdrawals (W), incompletes (I), repeats (RP), and failures (F). See section 4 for more details on different types of grades. There are two Withdrawal periods per term. The first period starts after 4:00 PM Friday, Week 2 to 4:00 PM Friday, Week 4. During this time, students can request to withdraw from the course without any consequence. The second period starts after 4:00 PM Friday, Week 4 to 4:00 PM Friday, Week 8. Attempted credits and a "W" grade will be recorded on the transcript for the course(s) from which students withdraw in this period. This grade does not affect a student’s GPA. In order to withdraw from a course, students need to submit the Course Withdrawal form to the Registrar’s Office, following the above timeline. A student who withdraws from a course beyond the halfway point of that course (end of week 4 for an 8-week course or end of week 8 for a full-term course) automatically receives a grade of ‘F’, which will affect the student’s GPA. Exceptions will only be permitted in exceptional circumstances (see Section 12), and it requires supporting documentation as well as the authorization of course instructor(s), the student’s academic/major advisor, and the Registrar’s Office. Question: Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?'})

prompts_3.append({'role': 'user', 'content': 'Context: All majors at Fulbright offer an optional capstone pathway to be completed during the student’s final year. Students may apply for the opportunity to fulfill 8 credits of their major requirements via a capstone project. A student wishing to do a capstone must apply during the term prior to the first term of the capstone (Capstone I), and may only apply after having completed 80 credits of coursework. Students who do not do a capstone will need to fulfill 8 credits of coursework as specified in their major. Individual majors will determine eligibility requirements, and students should discuss their proposals with program coordinators and potential advisors to prepare their applications. Capstone projects will be graded No Pass/Pass/Honors and must be evaluated by the supervising faculty at least one month before the end of the student’s graduating term. Students whose capstone is graded Honors will be awarded Graduation with Honors. Question: What are the consequences if I withdraw from capstone II?'})

prompts_3.append({'role': 'user', 'content': 'Context: Fulbright follows MOET regulations, which state that the maximum time to complete the program cannot exceed double the standard duration. In other words, students have a maximum of 8 years to complete their studies. Question: Can I continue my study for more than 4 years with financial aid?'})

prompts_3.append({'role': 'user', 'content': 'Context: For Applied Mathematics, These are the courses from other majors that emphasize the applications of Mathematics.  Students will have to take at least 2 applied courses (at least one at 200 level). If there is any prerequisite, students need to either complete the course’s prerequisites or obtain the instructor’s approval. Students pursuing a double major must complete all requirements for each major. If two majors (or two minors, or a major and a minor) share cross-listed or tagged courses, up to 8 credits (two courses) can be used to satisfy both sets of requirements. This limit is per each pair of majors (or major and minor) and does not restrict the total number of double-counted credits or courses a student can claim. For example, a student can major in both A and B and minor in C with a total of 6 double-counted courses: two in A and B, two in B and C, and two in A and C. This rule does not apply to courses that are not cross-listed or tagged. For instance, if major A requires certain courses from major B for out-of-area foundation, exploration, or applications of the major, and these courses are not cross-listed between A and B, they do not count toward the 8-credit limit. Question: Can you differentiate a cross-listed course and elective applied course in the case of a double major?'})


prompts_3.append({'role': 'user', 'content': """Context: All majors at Fulbright offer an optional capstone pathway to be completed during the student’s final year. Students may apply for the opportunity to fulfill 8 credits of their major requirements via a capstone project. A student wishing to do a capstone must apply during the term prior to the first term of the capstone (Capstone I), and may only apply after having completed 80 credits of coursework. Students who do not do a capstone will need to fulfill 8 credits of coursework as specified in their major. Individual majors will determine eligibility requirements, and students should discuss their proposals with program coordinators and potential advisors to prepare their applications. Capstone projects will be graded No Pass/Pass/Honors and must be evaluated by the supervising faculty at least one month before the end of the student’s graduating term. Students whose capstone is graded Honors will be awarded Graduation with Honors. Attempted credits and a "W" grade will be recorded on the transcript for the course(s) from which students withdraw in this period. This grade does not affect a student’s GPA. In order to withdraw from a course, students need to submit the Course Withdrawal form to the Registrar’s Office, following the above timeline. A student who withdraws from a course beyond the halfway point of that course (end of week 4 for an 8-week course or end of week 8 for a full-term course) automatically receives a grade of ‘F’, which will affect the student’s GPA. Question: I would like to ask for the capstone withdrawal policy. What will be the impact when I choose to drop the capstone before the Fall 2025 term starts? Will there be any penalties associated with withdrawing, such as a 'W' notation on my transcript?"""})

prompts_3.append({'role': 'user', 'content': 'Context: Econometrics: This course is concerned with the application of statistical theory to the analysis of economic data and the estimation of economic relationships. The course focuses on regression analysis and its uses in empirical economic research. Students will learn how to construct economic models and test them with data. Microeconomic Analysis: This course focuses on how incentives both constrain and direct the decision making of consumers, producers, and governments. Students will learn to use both graphical and optimization techniques to solve the problems faced by consumers (what to buy), producers (what to produce and what price to sell it at), and governments (which policies to enact). Macroeconomic Analysis: In this course, students will combine empirical observations and economic models to study the dynamics of the aggregate economy. This course focuses on the macroeconomic tools of government – fiscal and monetary policy – and their effects on long-run economic growth, employment, and inflation. Advanced level courses in Economics can be taken after Microeconomic and Macroeconomic Analysis and Econometrics. Financial Economics is counted as an advanced course. Question: I have taken Microeconomic Analysis and Macroeconomic Analysis in Fall 2023. Am I eligible to take Financial Economics in Spring 2024?'})

# Qwen model

## Model Loading

In [5]:
model_id_qwen_25_7B = "Qwen/Qwen2.5-7B-Instruct-1M"

pipeline_qwen_25_72B_text_gen = transformers.pipeline(
    "text-generation",
    model=model_id_qwen_25_7B,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cpu


## Question-answering

In [11]:
#Qwene 7B-1M
for question in prompts_1:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_qwen_25_72B_text_gen(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Qwen 2.5 with 7B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Qwen 2.5 with 7B is: 
{'role': 'assistant', 'content': "Withdrawal from a course after the fourth week typically does not result in a direct deduction of credits from the total covered by financial aid. However, it can affect your financial aid eligibility depending on your school's specific policies regarding the Return to Title IV (R2T4) policy.\n\nThe R2T4 policy calculates the amount of financial aid that you have earned based on the percentage of the term you completed. If you withdraw after the fourth week, you may be considered to have completed a certain percentage of the term, and the amount of aid you are eligible to keep is recalculated. This could potentially lead to a reduction in the amount of aid you receive for the term, but it does not directly deduct credits from your total covered by financial aid.\n\nFor precise information, you should consult your institution's financial aid office or review your school's specific R2T4 policy."}
----------------------

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


The answer by Qwen 2.5 with 7B is: 
{'role': 'assistant', 'content': 'The ability to continue your studies for more than four years with financial aid depends on several factors, including the type of financial aid you are receiving, your academic progress, and the policies of your institution. Here are some general points to consider:\n\n1. **Federal Financial Aid**: Federal financial aid, such as Pell Grants and Direct Loans, is typically designed for students to complete their degree within a standard time frame, which is usually six years for a bachelor’s degree. However, some students may be eligible for extensions under certain circumstances.\n\n2. **Institutional Aid**: Check with your institution’s financial aid office for specific policies regarding extended enrollment. Some schools may offer additional aid for students who need more time to complete their degree.\n\n3. **Academic Standing**: Maintaining good academic standing is crucial. If you are not making satisfactory aca

In [12]:
#Qwene 7B-1M
for question in prompts_2:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_qwen_25_72B_text_gen(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Qwen 2.5 with 7B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Qwen 2.5 with 7B is: 
{'role': 'assistant', 'content': 'Withdrawals from a course after the fourth week typically do not result in a direct deduction of credits from the total covered by financial aid, but they can affect the calculation of satisfactory academic progress (SAP) and eligibility for financial aid in other ways. \n\nFinancial aid institutions generally use the "aid eligibility" period, which is often the first 60% of the semester, to determine how much aid a student is eligible for. If a student withdraws after the fourth week, they may be considered to have completed a portion of the semester, and the amount of aid they are eligible for may be prorated based on the percentage of the semester they attended.\n\nFor specific policies, you should refer to the financial aid section of your institution\'s policy manual or contact the financial aid office directly, as policies can vary between institutions.'}
--------------------------------------------------------

In [13]:
#Qwene 7B-1M
for question in prompts_3:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_qwen_25_72B_text_gen(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Qwen 2.5 with 7B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Qwen 2.5 with 7B is: 
{'role': 'assistant', 'content': 'Withdrawal from a course after the 4th week but before the halfway point of the course (end of Week 4 for an 8-week course or end of Week 8 for a full-term course) will result in the course being recorded with a "W" grade on the transcript. This "W" grade does not affect the student’s GPA. However, the course will still count as attempted credits, and therefore, it will still be covered by financial aid.\n\nIf a student withdraws from a course after the halfway point of the course, they will receive an "F," which does affect the GPA and will also be counted as an attempted credit for financial aid purposes.\n\nThere is no automatic deduction of 4 credits in the total credits covered by financial aid for withdrawing from a course after the 4th week. The financial aid eligibility is determined based on the total attempted and earned credits, not the timing of the withdrawal itself.'}
-----------------------------------

In [14]:
# model_qwen_25_72B = "Qwen/Qwen2.5-72B-Instruct"
# pipeline_qwen_25_72B = transformers.pipeline(
#     "text-generation",
#     model=model_qwen_25_72B,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )

In [ ]:
# pipeline_qwen_25_72B_QA = transformers.pipeline(
#     "question-answering",
#     model="distilbert/distilbert-base-cased-distilled-squad",
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="cpu",
# )

# Llama-3.1-Tulu-3-8B

## Model loading

In [17]:
torch.cuda.empty_cache()

In [18]:
model_llama_31_tulu_8B = "allenai/Llama-3.1-Tulu-3-8B"
pipeline_llama_31_tulu_8B = transformers.pipeline(
    "text-generation",
    model=model_llama_31_tulu_8B,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


## Question-answering

In [19]:
#Qwene 7B-1M
for question in prompts_1:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_31_tulu_8B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.1 Tulu 8B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Llama 3.1 Tulu 8B is: 
{'role': 'assistant', 'content': "In many academic institutions, withdrawing from a course after a specific deadline, often the 4th week of the term, can indeed impact financial aid. However, the exact policy can vary significantly from one institution to another. Generally, withdrawing from a course after this point may result in a reduction in the total credits covered by financial aid, potentially leading to a decrease in the amount of financial aid received. This is because financial aid is typically calculated based on the number of credits a student is enrolled in at the time of disbursement.\n\nHere are some key points to consider:\n\n1. **Institutional Policy**: Check the specific policy of your institution regarding withdrawal and financial aid. Some schools may have a sliding scale where the more credits you withdraw from, the greater the impact on your financial aid.\n\n2. **Satisfactory Academic Progress (SAP)**: Withdrawing from courses

In [20]:
#Qwene 7B-1M
for question in prompts_2:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_31_tulu_8B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.1 Tulu 8B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Llama 3.1 Tulu 8B is: 
{'role': 'assistant', 'content': "Answer: No, withdrawing from a course after the 4th week typically does not result in a 4-credit deduction in the total credits covered by financial aid. The impact of a withdrawal on financial aid eligibility depends on the institution's specific policies and the timing of the withdrawal relative to the academic calendar and financial aid disbursement schedule. Generally, federal financial aid regulations allow students to withdraw from courses and still retain eligibility for aid, but institutions may have their own policies regarding the use of financial aid funds after a certain point in the term. It's important for students to consult their institution's financial aid office for precise information on how withdrawals affect their financial aid package. Always refer to the institution's official academic and financial aid policy documents for the most accurate and applicable information."}
----------------------

In [21]:
#Qwene 7B-1M
for question in prompts_3:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_31_tulu_8B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.1 Tulu 8B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

The answer by Llama 3.1 Tulu 8B is: 
{'role': 'assistant', 'content': 'No, withdrawal from a course after the 4th week does not result in a 4 credits deduction in the total credits covered by financial aid. When a student withdraws from a course after the drop/add period, the course remains on the transcript with a "W" grade, which does not affect the GPA. Financial aid calculations typically consider the student\'s enrollment status (full-time, part-time, etc.) based on the number of credits attempted, not the number of credits successfully completed or grades received. Therefore, withdrawing from a course does not directly impact the total credits covered by financial aid. However, it may affect the student\'s overall credit load for the term, which could impact eligibility for certain types of financial aid. Students should consult with the Financial Aid Office for specific details on how course withdrawals might affect their financial aid package.'}
--------------------------------

### Question-answering

In [36]:
for question in prompts:
    print(question['content'])
    context = """
    There are two Withdrawal periods per term. The first period starts after 4:00 PM Friday, Week 2 to 4:00 PM Friday, Week 4. During this time, students can request to withdraw from the course without any consequence. The second period starts after 4:00 PM Friday, Week 4 to 4:00 PM Friday, Week 8. Attempted credits and a “W” grade will be recorded on the transcript for the course(s) from which students withdraw in this period. This grade does not affect a student’s GPA. In order to withdraw from a course, students need to submit the Course Withdrawal form to the Registrar’s Office, following the above timeline. A student who withdraws from a course beyond the halfway point of that course (end of week 4 for an 8-week course or end of week 8 for a full-term course) automatically receives a grade of ‘F’, which will affect the student’s GPA. Exceptions will only be permitted in exceptional circumstances (see Section 12), and it requires supporting documentation as well as the authorization of course instructor(s), the student’s academic/major advisor, and the Registrar’s Office. In the case of approved exceptional circumstances, tuition credit will be considered and follow the “Fees & Charges Guide” for undergraduate students. No reduction in tuition is granted to students whose course withdrawal results in fewer than 12 academic credits. Students should be aware that course withdrawals may affect their financial aid eligibility if resulting in fewer than the minimum credits for full-time enrollment.
    """
    outputs = pipeline_qwen_25_72B_QA(question = "Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?", context = context, doc_stride = 370, max_answer_len = 370)
    print(outputs)

Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?
{'score': 0.22592197358608246, 'start': 1359, 'end': 1522, 'answer': 'Students should be aware that course withdrawals may affect their financial aid eligibility if resulting in fewer than the minimum credits for full-time enrollment'}
What are the consequences if I withdraw from capstone II?
{'score': 0.22592197358608246, 'start': 1359, 'end': 1522, 'answer': 'Students should be aware that course withdrawals may affect their financial aid eligibility if resulting in fewer than the minimum credits for full-time enrollment'}
Can I continue my study for more than 4 years with financial aid?
{'score': 0.22592197358608246, 'start': 1359, 'end': 1522, 'answer': 'Students should be aware that course withdrawals may affect their financial aid eligibility if resulting in fewer than the minimum credits for full-time enrollment'}
Can you differentiate a cross-listed course 

# Llama 3.3 70B

## Model loading

In [4]:
torch.cuda.empty_cache()

In [6]:
model_llama_33_70B = "meta-llama/Llama-3.3-70B-Instruct"
pipeline_llama_33_70B = transformers.pipeline(
    "text-generation",
    model=model_llama_33_70B,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


In [9]:
#Qwene 7B-1M
for question in prompts_1:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_33_70B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.3 70B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

KeyboardInterrupt: 

In [10]:
#Qwene 7B-1M
for question in prompts_2:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_33_70B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.3 70B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)

KeyboardInterrupt: 

In [ ]:
#Qwene 7B-1M
for question in prompts_2:
    questions = []
    questions.append(system_role)
    questions.append(question)
    outputs = pipeline_llama_33_70B(questions, max_new_tokens = 1024, num_beams = 2)
    print("The answer by Llama 3.3 70B is: ")
    print(outputs[0]['generated_text'][-1])
    print("-"*100)